In [2]:
import librosa 
import numpy as np
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [3]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../dataset/audio/'

#metadata = pd.read_csv('../dataset/dataset.csv')
metadata = pd.read_csv('../dataset/datasetM2.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  207  files


In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

Using TensorFlow backend.


In [5]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [6]:
### store the preprocessed data for use in the next notebook

%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)


In [7]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [10]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [11]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 350
num_batch_size = 40

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 165 samples, validate on 42 samples
Epoch 1/350
165/165 [==============================] - 0s 1ms/step - loss: 50.9235 - accuracy: 0.2970 - val_loss: 48.4105 - val_accuracy: 0.4524

Epoch 00001: val_loss improved from inf to 48.41048, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/350
165/165 [==============================] - 0s 68us/step - loss: 42.9902 - accuracy: 0.3939 - val_loss: 33.1384 - val_accuracy: 0.4524

Epoch 00002: val_loss improved from 48.41048 to 33.13845, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/350
165/165 [==============================] - 0s 68us/step - loss: 35.1874 - accuracy: 0.3636 - val_loss: 10.2919 - val_accuracy: 0.4524

Epoch 00003: val_loss improved from 33.13845 to 10.29185, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/350
165/165 [==============================] - 0s 64us/step - loss: 31.9448 - accuracy: 0.3394 - val_loss: 10.0452 - val_accuracy: 0.4524

Epoch 00004: val_loss improv

165/165 [==============================] - 0s 54us/step - loss: 2.6237 - accuracy: 0.4606 - val_loss: 1.0329 - val_accuracy: 0.5000

Epoch 00037: val_loss did not improve from 1.00297
Epoch 38/350
165/165 [==============================] - 0s 55us/step - loss: 2.6915 - accuracy: 0.4909 - val_loss: 1.0326 - val_accuracy: 0.5238

Epoch 00038: val_loss did not improve from 1.00297
Epoch 39/350
165/165 [==============================] - 0s 54us/step - loss: 2.3499 - accuracy: 0.4788 - val_loss: 1.0330 - val_accuracy: 0.5000

Epoch 00039: val_loss did not improve from 1.00297
Epoch 40/350
165/165 [==============================] - 0s 55us/step - loss: 2.3829 - accuracy: 0.4727 - val_loss: 1.0338 - val_accuracy: 0.5000

Epoch 00040: val_loss did not improve from 1.00297
Epoch 41/350
165/165 [==============================] - 0s 54us/step - loss: 2.2357 - accuracy: 0.5030 - val_loss: 1.0336 - val_accuracy: 0.5000

Epoch 00041: val_loss did not improve from 1.00297
Epoch 42/350
165/165 [======


Epoch 00078: val_loss did not improve from 1.00297
Epoch 79/350
165/165 [==============================] - 0s 60us/step - loss: 1.2796 - accuracy: 0.4788 - val_loss: 1.0393 - val_accuracy: 0.4048

Epoch 00079: val_loss did not improve from 1.00297
Epoch 80/350
165/165 [==============================] - 0s 57us/step - loss: 1.3073 - accuracy: 0.4606 - val_loss: 1.0395 - val_accuracy: 0.4048

Epoch 00080: val_loss did not improve from 1.00297
Epoch 81/350
165/165 [==============================] - 0s 54us/step - loss: 1.3362 - accuracy: 0.4788 - val_loss: 1.0396 - val_accuracy: 0.4048

Epoch 00081: val_loss did not improve from 1.00297
Epoch 82/350
165/165 [==============================] - 0s 55us/step - loss: 1.1948 - accuracy: 0.4970 - val_loss: 1.0397 - val_accuracy: 0.4048

Epoch 00082: val_loss did not improve from 1.00297
Epoch 83/350
165/165 [==============================] - 0s 56us/step - loss: 1.1906 - accuracy: 0.5273 - val_loss: 1.0398 - val_accuracy: 0.4048

Epoch 00083: v

165/165 [==============================] - 0s 56us/step - loss: 1.1552 - accuracy: 0.5455 - val_loss: 1.0409 - val_accuracy: 0.4286

Epoch 00120: val_loss did not improve from 1.00297
Epoch 121/350
165/165 [==============================] - 0s 76us/step - loss: 1.1636 - accuracy: 0.5030 - val_loss: 1.0399 - val_accuracy: 0.4286

Epoch 00121: val_loss did not improve from 1.00297
Epoch 122/350
165/165 [==============================] - 0s 64us/step - loss: 1.1618 - accuracy: 0.4788 - val_loss: 1.0387 - val_accuracy: 0.4286

Epoch 00122: val_loss did not improve from 1.00297
Epoch 123/350
165/165 [==============================] - 0s 56us/step - loss: 1.2810 - accuracy: 0.4909 - val_loss: 1.0378 - val_accuracy: 0.4286

Epoch 00123: val_loss did not improve from 1.00297
Epoch 124/350
165/165 [==============================] - 0s 55us/step - loss: 1.1718 - accuracy: 0.5030 - val_loss: 1.0372 - val_accuracy: 0.4286

Epoch 00124: val_loss did not improve from 1.00297
Epoch 125/350
165/165 [=

165/165 [==============================] - 0s 54us/step - loss: 1.1159 - accuracy: 0.5515 - val_loss: 1.0404 - val_accuracy: 0.4286

Epoch 00161: val_loss did not improve from 1.00297
Epoch 162/350
165/165 [==============================] - 0s 53us/step - loss: 1.0778 - accuracy: 0.5152 - val_loss: 1.0401 - val_accuracy: 0.4286

Epoch 00162: val_loss did not improve from 1.00297
Epoch 163/350
165/165 [==============================] - 0s 55us/step - loss: 1.0790 - accuracy: 0.5394 - val_loss: 1.0363 - val_accuracy: 0.4762

Epoch 00163: val_loss did not improve from 1.00297
Epoch 164/350
165/165 [==============================] - 0s 54us/step - loss: 1.1562 - accuracy: 0.4848 - val_loss: 1.0364 - val_accuracy: 0.4762

Epoch 00164: val_loss did not improve from 1.00297
Epoch 165/350
165/165 [==============================] - 0s 55us/step - loss: 1.1520 - accuracy: 0.5091 - val_loss: 1.0339 - val_accuracy: 0.4762

Epoch 00165: val_loss did not improve from 1.00297
Epoch 166/350
165/165 [=

165/165 [==============================] - 0s 53us/step - loss: 1.0411 - accuracy: 0.5455 - val_loss: 0.9972 - val_accuracy: 0.4762

Epoch 00200: val_loss did not improve from 0.97175
Epoch 201/350
165/165 [==============================] - 0s 54us/step - loss: 1.0466 - accuracy: 0.5091 - val_loss: 0.9974 - val_accuracy: 0.4762

Epoch 00201: val_loss did not improve from 0.97175
Epoch 202/350
165/165 [==============================] - 0s 54us/step - loss: 1.0461 - accuracy: 0.5333 - val_loss: 0.9971 - val_accuracy: 0.4762

Epoch 00202: val_loss did not improve from 0.97175
Epoch 203/350
165/165 [==============================] - 0s 54us/step - loss: 1.0602 - accuracy: 0.5394 - val_loss: 0.9967 - val_accuracy: 0.4762

Epoch 00203: val_loss did not improve from 0.97175
Epoch 204/350
165/165 [==============================] - 0s 55us/step - loss: 1.1242 - accuracy: 0.5697 - val_loss: 0.9928 - val_accuracy: 0.4762

Epoch 00204: val_loss did not improve from 0.97175
Epoch 205/350
165/165 [=


Epoch 00237: val_loss did not improve from 0.90741
Epoch 238/350
165/165 [==============================] - 0s 60us/step - loss: 1.0738 - accuracy: 0.5394 - val_loss: 0.9242 - val_accuracy: 0.4762

Epoch 00238: val_loss did not improve from 0.90741
Epoch 239/350
165/165 [==============================] - 0s 57us/step - loss: 1.0260 - accuracy: 0.5091 - val_loss: 0.9227 - val_accuracy: 0.5000

Epoch 00239: val_loss did not improve from 0.90741
Epoch 240/350
165/165 [==============================] - 0s 60us/step - loss: 0.9879 - accuracy: 0.5697 - val_loss: 0.9162 - val_accuracy: 0.5476

Epoch 00240: val_loss did not improve from 0.90741
Epoch 241/350
165/165 [==============================] - 0s 71us/step - loss: 0.9745 - accuracy: 0.6061 - val_loss: 0.9103 - val_accuracy: 0.5714

Epoch 00241: val_loss did not improve from 0.90741
Epoch 242/350
165/165 [==============================] - 0s 75us/step - loss: 0.9823 - accuracy: 0.6061 - val_loss: 0.8953 - val_accuracy: 0.6190

Epoch 002

165/165 [==============================] - 0s 66us/step - loss: 0.8683 - accuracy: 0.6303 - val_loss: 0.7799 - val_accuracy: 0.6667

Epoch 00273: val_loss did not improve from 0.75154
Epoch 274/350
165/165 [==============================] - 0s 72us/step - loss: 0.9066 - accuracy: 0.6182 - val_loss: 0.7687 - val_accuracy: 0.6667

Epoch 00274: val_loss did not improve from 0.75154
Epoch 275/350
165/165 [==============================] - 0s 64us/step - loss: 0.8934 - accuracy: 0.6788 - val_loss: 0.7619 - val_accuracy: 0.6667

Epoch 00275: val_loss did not improve from 0.75154
Epoch 276/350
165/165 [==============================] - 0s 57us/step - loss: 0.9478 - accuracy: 0.5758 - val_loss: 0.7502 - val_accuracy: 0.6667

Epoch 00276: val_loss improved from 0.75154 to 0.75020, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 277/350
165/165 [==============================] - 0s 59us/step - loss: 0.9192 - accuracy: 0.6485 - val_loss: 0.7382 - val_accuracy: 0.7143

Epoch 00277: 

165/165 [==============================] - 0s 55us/step - loss: 0.7593 - accuracy: 0.6485 - val_loss: 0.6511 - val_accuracy: 0.7143

Epoch 00310: val_loss improved from 0.65595 to 0.65113, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 311/350
165/165 [==============================] - 0s 62us/step - loss: 0.7529 - accuracy: 0.6848 - val_loss: 0.6497 - val_accuracy: 0.6905

Epoch 00311: val_loss improved from 0.65113 to 0.64966, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 312/350
165/165 [==============================] - 0s 62us/step - loss: 0.8599 - accuracy: 0.6182 - val_loss: 0.6502 - val_accuracy: 0.6905

Epoch 00312: val_loss did not improve from 0.64966
Epoch 313/350
165/165 [==============================] - 0s 57us/step - loss: 0.8140 - accuracy: 0.6848 - val_loss: 0.6393 - val_accuracy: 0.6905

Epoch 00313: val_loss improved from 0.64966 to 0.63933, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 314/350
165/165 [=============


Epoch 00349: val_loss did not improve from 0.61535
Epoch 350/350
165/165 [==============================] - 0s 59us/step - loss: 0.6950 - accuracy: 0.6848 - val_loss: 0.5994 - val_accuracy: 0.7143

Epoch 00350: val_loss improved from 0.61535 to 0.59942, saving model to saved_models/weights.best.basic_mlp.hdf5
Training completed in time:  0:00:06.336345


In [12]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.800000011920929
Testing Accuracy:  0.7142857313156128


In [13]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [14]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [46]:
# test

filename = '../dataset/audio/alibaba4.wav' 
print_prediction(filename) 

The predicted class is: tap 

double_tap 		 :  0.00000000000014869849058671280595
long_tap 		 :  0.00000000000000000000000000000000
swipe 		 :  0.00000000000000000000000000967433
tap 		 :  1.00000000000000000000000000000000
zoom 		 :  0.00000000000000000000000000000000
